<a href="https://colab.research.google.com/github/dbtjr1103/Basic_practice/blob/main/AI%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DACON 유방암의 임파선 전이 예측 AI 경진대회 연습

https://dacon.io/competitions/official/236011/codeshare/7355?page=1&dtype=recent

In [4]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [5]:
!gdown --id 1q4d8FmF452uJ9SdlW1zQ2SMJYAphKwts

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1q4d8FmF452uJ9SdlW1zQ2SMJYAphKwts
To: /content/open.zip
100% 17.5G/17.5G [01:51<00:00, 156MB/s]


In [8]:
!unzip -q open.zip

In [6]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import cv2
import albumentations as A
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5665312748938981247
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13955993540348142574
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [9]:
# Data받기(수술연월일 = parse_date로 받기)
path = '/content/'
train = pd.read_csv(path + 'train.csv', parse_dates=["수술연월일"]).copy()
test = pd.read_csv(path + 'test.csv', parse_dates=["수술연월일"]).copy()
clinical_info = pd.read_excel(path + 'clinical_info.xlsx').copy()

In [10]:
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN,0
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN,1
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0,0
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN,0
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0


In [11]:
# 빈값 확인
train.isna().sum()

ID                     0
img_path               0
mask_path              0
나이                     0
수술연월일                  0
진단명                    0
암의 위치                  0
암의 개수                  0
암의 장경                 69
NG                    51
HG                    86
HG_score_1            92
HG_score_2            92
HG_score_3            89
DCIS_or_LCIS_여부        0
DCIS_or_LCIS_type    874
T_category             4
ER                     1
ER_Allred_score      297
PR                     1
PR_Allred_score      453
KI-67_LI_percent     235
HER2                  14
HER2_IHC              24
HER2_SISH            753
HER2_SISH_ratio      825
BRCA_mutation        945
N_category             0
dtype: int64

In [12]:
# column 이름 확인
train.columns.unique()

Index(['ID', 'img_path', 'mask_path', '나이', '수술연월일', '진단명', '암의 위치', '암의 개수',
       '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
       'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 'ER',
       'ER_Allred_score', 'PR', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2',
       'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation',
       'N_category'],
      dtype='object')

In [13]:
# 수술연월일 4개로 분리
def date_split(df):

    df['year'] = df['수술연월일'].dt.year
    df['month'] = df['수술연월일'].dt.month
    df['day'] = df['수술연월일'].dt.day
    df['quarter'] = df['수술연월일'].dt.quarter

    return df
    
train = date_split(train)
test = date_split(test)
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category,year,month,day,quarter
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,0.0,1.0,NaN,NaN,NaN,0,2015,10,23,4
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,0.0,0.0,NaN,NaN,NaN,1,2015,10,28,4
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,0.0,1.0,NaN,NaN,0.0,0,2015,10,29,4
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,1.0,3.0,NaN,NaN,NaN,0,2016,3,8,1
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,1.0,2.0,1.0,5.44,NaN,0,2015,10,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,0.0,0.0,NaN,NaN,NaN,1,2006,12,22,4
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,0.0,0.0,NaN,NaN,NaN,0,2006,11,17,4
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,0.0,0.0,NaN,NaN,NaN,1,2006,11,10,4
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,0.0,0.0,NaN,NaN,NaN,0,2006,9,22,3


In [14]:
# 암의장경 fillna
train['암의 장경'] = train['암의 장경'].fillna(train['암의 장경'].mean())
train = train.fillna(0)

test['암의 장경'] = test['암의 장경'].fillna(train['암의 장경'].mean())
test = test.fillna(0)

In [15]:
# label값 복사
label = train['N_category'].values.copy()

In [16]:
# ID, mask_path 제거 후 각각 train_table, test table로 저장
ignore_cols_train = ['ID', 'mask_path', '수술연월일', 'N_category']
ignore_cols_test = ['ID', 'img_path', '수술연월일']

train_table = train.drop(ignore_cols_train, axis = 1).copy()
test_table = test.drop(ignore_cols_test, axis = 1).copy()
train_table

,img_path,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,year,month,day,quarter
0,./train_imgs/BC_01_0001.png,63,1,2,1,19.000000,2.0,1.0,2.0,2.0,...,12.0,0.0,1.0,0.0,0.00,0.0,2015,10,23,4
1,./train_imgs/BC_01_0002.png,51,1,1,1,22.000000,3.0,3.0,3.0,3.0,...,70.0,0.0,0.0,0.0,0.00,0.0,2015,10,28,4
2,./train_imgs/BC_01_0003.png,37,1,2,1,18.375564,2.0,0.0,0.0,0.0,...,7.0,0.0,1.0,0.0,0.00,0.0,2015,10,29,4
3,./train_imgs/BC_01_0004.png,54,1,2,1,0.000000,3.0,3.0,3.0,3.0,...,1.0,1.0,3.0,0.0,0.00,0.0,2016,3,8,1
4,./train_imgs/BC_01_0005.png,57,1,2,1,8.000000,2.0,2.0,3.0,2.0,...,8.0,1.0,2.0,1.0,5.44,0.0,2015,10,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,./train_imgs/BC_01_3464.png,65,1,2,1,25.000000,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,12,22,4
996,./train_imgs/BC_01_3482.png,48,1,1,1,7.000000,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,11,17,4
997,./train_imgs/BC_01_3485.png,64,1,2,1,15.000000,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,11,10,4
998,./train_imgs/BC_01_3502.png,50,1,1,1,7.000000,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,9,22,3


In [17]:
# train_test_split으로 val 데이터 생성 (성능 체크 목적)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_table, label, random_state = 42)

In [18]:
X_train

,img_path,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,year,month,day,quarter
82,./train_imgs/BC_01_0404.png,72,1,2,1,13.0,2.0,1.0,2.0,2.0,...,6.38,0.0,1.0,0.0,0.00,0.0,2019,12,26,4
991,./train_imgs/BC_01_3416.png,69,1,1,1,9.0,1.0,1.0,3.0,1.0,...,0.00,0.0,1.0,0.0,0.00,0.0,2009,5,6,2
789,./train_imgs/BC_01_2899.png,40,1,2,1,18.0,2.0,1.0,2.0,1.0,...,10.00,1.0,2.0,0.0,0.00,0.0,2008,5,27,2
894,./train_imgs/BC_01_3040.png,61,1,1,1,12.0,2.0,2.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,4,9,2
398,./train_imgs/BC_01_1939.png,40,1,1,1,1.5,1.0,1.0,3.0,1.0,...,5.00,0.0,1.0,0.0,0.00,0.0,2015,9,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,./train_imgs/BC_01_0530.png,54,1,2,1,15.0,3.0,3.0,3.0,3.0,...,24.15,0.0,2.0,0.0,1.44,0.0,2019,1,22,1
270,./train_imgs/BC_01_1252.png,56,1,1,1,22.0,3.0,2.0,2.0,3.0,...,11.71,0.0,2.0,0.0,1.90,0.0,2019,7,24,3
860,./train_imgs/BC_01_2997.png,62,1,2,1,12.0,3.0,2.0,3.0,3.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,7,20,3
435,./train_imgs/BC_01_2101.png,63,1,2,1,14.0,2.0,2.0,3.0,2.0,...,20.00,0.0,0.0,0.0,0.00,0.0,2015,3,13,1


In [19]:
# train에 있는 img_path만 따로 저장(각각 X_train_img 와 X_val_img)
# 현재 dataset은 코드저장되어있는 부분과 다른 곳에 위치하므로, path수정후 저장
X_train_img = X_train['img_path'].values.copy()
X_train_img = [path + j[2:] for j in X_train_img]
X_train_img = np.reshape(X_train_img, (len(X_train_img)))

X_val_img = X_val['img_path'].values.copy()
X_val_img = [path + j[2:] for j in X_val_img]
X_val_img = np.reshape(X_val_img, (len(X_val_img)))

In [20]:
# img_path 따로 뺐으니, X_train, X_val에서 img_path 제거
ignore_img = ['img_path']

X_train = X_train.drop(ignore_img, axis = 1)
X_val = X_val.drop(ignore_img, axis = 1)

In [22]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
# XGB Classifier
from xgboost import XGBRFClassifier

xgb = XGBRFClassifier(random_state = 42)
xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)])

[0]	validation_0-error:0.196


XGBRFClassifier(random_state=42)

In [29]:
# test_table 예측 확인
xgb.predict(test_table)

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
# test_table 예측 한 확률을 각각 출력 후 저장
# 예측 확률로 한 이유: 이후 각 예측 확률을 더해 최종 예측을 만들기 위해 
xgb_pred = xgb.predict_proba(test_table)

In [31]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 54 kB/s 


In [32]:
# CatboostClassifier
from catboost import CatBoostClassifier

cat = CatBoostClassifier(random_state = 42, verbose = 0)
cat.fit(X_train, y_train, eval_set = [(X_val, y_val)])

In [33]:
cat.predict(test_table)

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

In [34]:
# 예측 확률 저장
cat_pred = cat.predict_proba(test_table)

In [35]:
# ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier

ex_clf = ExtraTreesClassifier(random_state = 42)
ex_clf.fit(X_train, y_train)

ExtraTreesClassifier(random_state=42)

In [36]:
ex_clf.predict(test_table)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

In [37]:
ex_pred = ex_clf.predict_proba(test_table)

In [38]:
# RandomForestClassifier  
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [39]:
rf.predict(test_table)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

In [40]:
rf_pred = rf.predict_proba(test_table)

In [41]:
# LGBMClassifier
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state = 42)
lgbm.fit(X_train, y_train)

LGBMClassifier(random_state=42)

In [43]:
lgbm.predict(test_table)

array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

In [44]:
lgbm_pred = lgbm.predict_proba(test_table)

In [45]:
# f1 score 확인
# 성능 확인 방법 : model에 넣지 않은 val을 넣고 성능 확인
from sklearn.metrics import f1_score
def scoring(y, pred):
    score = f1_score(y_true = y, y_pred = pred)
    print(score)

In [46]:
model = [xgb, cat, rf, ex_clf, lgbm]
for i in model:
    print(str(i))
    print(i.score(X_val, y_val))

XGBRFClassifier(random_state=42)
0.804
0.828
RandomForestClassifier(random_state=42)
0.82
ExtraTreesClassifier(random_state=42)
0.812
LGBMClassifier(random_state=42)
0.812


In [47]:
for i in model:
    print(str(i))
    print(scoring(y_val, i.predict(X_val)))

XGBRFClassifier(random_state=42)
0.7932489451476793
None
0.8326848249027238
None
RandomForestClassifier(random_state=42)
0.8221343873517787
None
ExtraTreesClassifier(random_state=42)
0.8142292490118578
None
LGBMClassifier(random_state=42)
0.8212927756653993
None


In [48]:
# Extra는 RF와 비슷하므로 둘 중 낮은 값인 Extra는 사용 X
# Image모델(InceptionV3 사용)
# test의 img_path 역시 dataset이 위치한 곳으로 수정 후 따로 저장
test_img = test['img_path'].values.copy()
test_img = [path + j[2:] for j in test_img]
test_img = np.reshape(test_img, (len(test_img)))

In [49]:
# 잘 수정되었는지 확인
X_train_img[0]

'/content/train_imgs/BC_01_0404.png'

In [50]:
# img_size 지정 img_resize
img_size = 512
img_channel = 3
batch = 8

resize_crop = A.Compose([
    A.Resize(img_size, img_size, interpolation = cv2.INTER_AREA)
])

In [52]:
# CustomSequence제작
# 메모리를 위해 dataset 사용했다고 함
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
from tensorflow.keras.applications.xception import preprocess_input
import math

class CustomSequence(Sequence):
    def __init__(self, filenames, target, batch_size, aug=None, shuffle=False):
        # image의 절대경로들
        self.filenames = filenames
        self.target = target
        self.batch_size = batch_size
        # albumentation 객체
        self.aug = aug
        self.shuffle = shuffle
        
        # 훈련 데이터의 경우
        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        # 총 step의 갯수
        return math.ceil(len(self.filenames) / self.batch_size)

    def __getitem__(self, index):
        # 현재 인덱스를 기준으로 batch_size만큼 데이터를 가져옴
        meta_data = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # 훈련, 검증 데이터세트인 경우
        if self.target is not None:
            label_batch = self.target[index*self.batch_size:(index+1)*self.batch_size]

        # 불러온 meta_data를 np.array로 저장할 빈 공간을 생성
        image_batch = np.zeros((meta_data.shape[0], img_size, img_size, img_channel))
        for i in range(meta_data.shape[0]):
            # cv2는 이미지를 BGR로 불러오기 때문이 RGB로 바꾸어줌
            image = cv2.cvtColor(cv2.imread(meta_data[i]), cv2.COLOR_BGR2RGB)
            # augmentation이 있으면 적용
            if self.aug is not None:
                image = self.aug(image=image)['image']

            # 이미지 값을 self.pre_func 함수로 스케일링
            image = image * 1./255

            # 빈 이미지 배치에 최종 이미지를 등록
            image_batch[i] = image
            
        if self.target is not None:
            return image_batch, label_batch
        else:
            return image_batch
        
    def on_epoch_end(self):
        # 파일과 라벨을 같이 섞어 주어야한다.
        if self.shuffle is True:
            self.image_filenames, self.target = shuffle(self.filenames, self.target)
        else:
            pass

In [53]:
train_dataset = CustomSequence(X_train_img, y_train, batch, resize_crop)
val_dataset = CustomSequence(X_val_img, y_val, batch, resize_crop)

In [54]:
len(train_dataset)

94

In [55]:
# dataset 확인용
def show_first_data(dataset, image_verbose=False):
    first_data = next(iter(dataset))
    images = first_data[0]
    labels = first_data[1]
    print(images.shape, labels.shape)
    print(images[0])
    if image_verbose:
        fig, axs = plt.subplots(figsize=(20, 8), nrows=1, ncols=4)
        for i in range(4):
            axs[i].imshow(np.array(images[i], dtype='int32'))

In [56]:
show_first_data(train_dataset)

(8, 512, 512, 3) (8,)
[[[0.95686275 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 [[0.95686275 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.96078431]
  [0.95686275 0.96078431 0.95686275]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 [[0.95686275 0.95686275 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.96078431]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 ...

 [[0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  ...
  [0.95294118 0.95686275 0.95294118]
  [0.95294118 0.95294118 0.95294118]
  [0.95294118 0.95294118 0.95294118]]

 [[0.95686275 0.95686275 0.95686275]
  [0.95686275 0

In [57]:
# CNN 모델 생성(inceptionV3)
def make_model():
    base_model = keras.applications.InceptionV3(include_top = False, weights = 'imagenet', input_shape = (img_size, img_size, 3), pooling = None)
    x = base_model.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(512, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(256, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(128, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(64, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    predictions = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.models.Model(inputs=base_model.input, outputs=predictions)    
    return model

In [58]:
img_model = make_model()

87910968/87910968 [==============================] - 5s 0us/step


In [59]:
img_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 255, 255, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 255, 255, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [60]:
# callback함수(ReduceLROnPlateau, EarlyStopping, Checkpoint)
# val로 성능 확인하면서 3번 이상 성능 감소 안하면 learning late감소, 10번 val 성능이 감소 안하면 정지, 가장 좋은 모델로 Callback)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode = 'min')
ES = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
checkpoint_cb = keras.callbacks.ModelCheckpoint('img_model.h5', save_best_only=True)

In [61]:
img_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

img_model.fit(train_dataset, batch_size = batch,
              validation_data = val_dataset,
              epochs=100, callbacks=[reduce_lr, ES, checkpoint_cb],
              verbose = 1)
# loss는 주는데, accuracy도 줄암, val_acc는 변화없음. 16까지 했다고 함. 

Epoch 1/100
94/94 [==============================] - 360s 4s/step - loss: 0.7915 - acc: 0.5387 - val_loss: 1286.8024 - val_acc: 0.5000 - lr: 0.0010
Epoch 2/100
94/94 [==============================] - 336s 4s/step - loss: 0.7227 - acc: 0.5040 - val_loss: 0.7352 - val_acc: 0.5000 - lr: 0.0010
Epoch 3/100
94/94 [==============================] - 337s 4s/step - loss: 0.7219 - acc: 0.4947 - val_loss: 0.6953 - val_acc: 0.5000 - lr: 0.0010
Epoch 4/100
94/94 [==============================] - 338s 4s/step - loss: 0.7324 - acc: 0.5107 - val_loss: 0.7028 - val_acc: 0.5000 - lr: 0.0010
Epoch 5/100
94/94 [==============================] - 338s 4s/step - loss: 0.7148 - acc: 0.4813 - val_loss: 0.7174 - val_acc: 0.4880 - lr: 0.0010
Epoch 6/100
94/94 [==============================] - ETA: 0s - loss: 0.7172 - acc: 0.5107
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
94/94 [==============================] - 337s 4s/step - loss: 0.7172 - acc: 0.5107 - val_loss: 0.7105 - va

KeyboardInterrupt: ignored

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
# test이미지 dataset으로 제작
test_dataset = CustomSequence(test_img, None, batch, resize_crop)

In [64]:
test_dataset

In [65]:
# CNN에서 predict를 하면, 0~1 사이의 값(확률)
# 이는 1로 될 확률과 같으므로, 0일때의 확률은 1-predict이고 1일때의 확률은 predict로 하는 array를 생성
def img_predict_proba(model, test_dataset):
    img_pred = model.predict(test_dataset)
    pred_test = np.zeros((len(img_pred), 2))

    for i in range(0, len(img_pred)):
        pred_test[i][0] = (1 - img_pred[i])
        pred_test[i][1] = img_pred[i]

    return pred_test

In [66]:
img_pred = img_predict_proba(img_model, test_dataset)

32/32 [==============================] - 86s 3s/step


In [67]:
# val_dataset으로 f1_score 확인
scoring(y_val, np.argmax(img_predict_proba(img_model, val_dataset), axis = 1))

32/32 [==============================] - 86s 3s/step
0.653409090909091


In [68]:
# xgb, rf, cat, lgbm과 img 확률들을 더해 최종 예측 생성(각각 0.2씩)
pred = (img_pred + xgb_pred + rf_pred + cat_pred + lgbm_pred) / 5
pred

array([[0.19574161, 0.80425839],
       [0.5475486 , 0.4524514 ],
       [0.85394119, 0.14605881],
       [0.66926424, 0.33073576],
       [0.27315857, 0.72684143],
       [0.19116867, 0.80883133],
       [0.22388468, 0.77611532],
       [0.45925778, 0.54074222],
       [0.39571678, 0.60428322],
       [0.32058547, 0.67941453],
       [0.51544057, 0.48455943],
       [0.41177576, 0.58822424],
       [0.56636827, 0.43363173],
       [0.38975969, 0.61024031],
       [0.74816934, 0.25183066],
       [0.17005255, 0.82994745],
       [0.17650311, 0.82349689],
       [0.80378746, 0.19621254],
       [0.17287151, 0.82712849],
       [0.29675455, 0.70324544],
       [0.32660063, 0.67339937],
       [0.17499244, 0.82500756],
       [0.85986397, 0.14013604],
       [0.1572438 , 0.8427562 ],
       [0.84041516, 0.15958484],
       [0.14977386, 0.85022614],
       [0.18131123, 0.81868877],
       [0.81702254, 0.18297745],
       [0.13837678, 0.86162322],
       [0.66378359, 0.33621641],
       [0.

In [69]:
# np.argmax를 이용해 확인
np.argmax(pred, axis = 1)

array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

In [70]:
# submission csv생성
output = pd.DataFrame({'ID' : test['ID'], 'N_category':np.argmax(pred, axis = 1)})
output.to_csv('/content/drive/MyDrive/final_school.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
